In [1]:
"""
Retrieve contents of a GDrive directory and ingest all files from there, one at a time.
All authorization related stuff is in credentials.json and token.json.
If the token is expired, remove and regenerate it using the other notebook.
"""

'\nRetrieve contents of a GDrive directory and ingest all files from there, one at a time.\nAll authorization related stuff is in credentials.json and token.json.\nIf the token is expired, remove and regenerate it using the other notebook.\n'

In [2]:
# Use loader for dirctory listing and reader for downloading the file locally
from googleapiclient.discovery import build
from llama_index.readers.google import GoogleDriveReader
from langchain_google_community import GoogleDriveLoader
from os import unlink, listdir

In [3]:
# retrieve all documents from the folder
folder_id = '1gGPeJKIPHLoG3LKReIii6lqcVzH5erI1'
# folder_id = '1iLrqxhq_ygOubgQaywfJOa8dqKiOeSpT'

loader = GoogleDriveLoader(
    folder_id=folder_id,

    # token generated separately in exp1.ipynb
    token_path="./token.json",

    # Optional: configure whether to recursively fetch files from subfolders. Defaults to False.
    recursive=True,
)

creds = loader._load_credentials()
service = build("drive", "v3", credentials=creds)
docs_info = loader._fetch_files_recursive(folder_id=folder_id, service=service)

In [4]:
print(len(docs_info))
info = docs_info[0]
print(info)
mime_type, file_id, file_name = info['mimeType'], info['id'], info['name']

461
{'mimeType': 'application/pdf', 'parents': ['1iLrqxhq_ygOubgQaywfJOa8dqKiOeSpT'], 'id': '1kU3VVPcWf6Dld1S49Av6JvDn4qbRd2x3', 'name': 'Iron & Steel Manufacturing in the US.pdf', 'trashed': False}


In [5]:
for info in docs_info:
    # print(info['name'])
    if info['name'].startswith('A'):
        print(info)

In [6]:
reader = GoogleDriveReader()
# hacky way to set credentials in the reader
reader._creds = creds
new_file_name = reader._download_file(file_id, file_name)
assert new_file_name in listdir()

NameError: name 'GoogleDriveReader' is not defined

In [7]:
# TODO: upload the file to Weaviate once the schema is ready.

In [8]:
unlink(file_name)